In [57]:
from pyspark.sql import SparkSession # import spark sessions
from pyspark.sql.types import IntegerType
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row
import os, sys
os.environ['PYSPARK_PYTHON'] = sys.executable # needed for spark to work with python, grabs the venv python executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [58]:
spark = SparkSession.builder.appName("homework").getOrCreate() # creates the app to use later for sql and to read the
rdd2 = spark.read.option("delimiter", ",").option("header", True).csv("social_net_with_header.csv") # read the csv file with headers enabled
rdd = rdd2.rdd # converts the dataframe to rdd

In [59]:
smart_K = rdd.sortBy(lambda row: -int(row[3])).take(5) # sorts the table by friends. Negative sign means desc and then take the top 5 results 
print(smart_K) # print the output

[Row(userID='106', name='Beverly', age='18', friends='499'), Row(userID='278', name='Data', age='44', friends='499'), Row(userID='353', name='Julian', age='64', friends='499'), Row(userID='192', name='Jadzia', age='45', friends='497'), Row(userID='266', name='Jean-Luc', age='66', friends='496')]


In [60]:
df_students_K = spark.read.option("delimiter", ",").option("header", True).csv("social_net_with_header.csv") # read the csv and make a dataframe
df_students_K = df_students_K.withColumn("friends",df_students_K['friends'].cast(IntegerType())) # to sort friends, we need to convert the numbers from str to int
df_smart_5 = df_students_K.sort(df_students_K.friends,ascending=False).take(5) # sort the dataframe column friends in desc order and take the top 5.
print(df_smart_5)

[Row(userID='106', name='Beverly', age='18', friends=499), Row(userID='278', name='Data', age='44', friends=499), Row(userID='353', name='Julian', age='64', friends=499), Row(userID='192', name='Jadzia', age='45', friends=497), Row(userID='266', name='Jean-Luc', age='66', friends=496)]


In [61]:
df_students_K.createOrReplaceTempView("studentsSQL") # create an app view for the sql table.

In [62]:
smart_sql_K = spark.sql("select * from studentsSQL order by friends desc limit 5") # select all columns and then order by friends desc with a limit of 5
smart_sql_K.show() # prints the results from the sql query.

+------+--------+---+-------+
|userID|    name|age|friends|
+------+--------+---+-------+
|   106| Beverly| 18|    499|
|   278|    Data| 44|    499|
|   353|  Julian| 64|    499|
|   192|  Jadzia| 45|    497|
|   266|Jean-Luc| 66|    496|
+------+--------+---+-------+

